In [ ]:
import re
import os
import ntpath
import numpy as np
import pandas as pd
from datetime import date, datetime, timedelta
import xarray as xr
import cartopy.crs as ccrs
from matplotlib import pyplot as plt

In [ ]:
data_vars = {0: ('PSNM:000', 'Pressão Reduzida ao Nível Médio do Mar [hPa]'),
 1: ('TEMP:850', 'Temperatura Absoluta @ 850 hPa [K]'),
 2: ('TEMP:500', 'Temperatura Absoluta @ 500 hPa [K]'),
 3: ('TEMP:250', 'Temperatura Absoluta @ 250 hPa [K]'),
 4: ('UMES:925', 'Umidade Específica @ 925 hPa [g/Kg]'),
 5: ('AGPL:925', 'Água Precipitável @ 925 hPa [Kg/m2]'),
 6: ('ZGEO:850', 'Altura Geopotencial @ 850 hPa [gpm]'),
 7: ('ZGEO:500', 'Altura Geopotencial @ 500 hPa [gpm]'),
 8: ('ZGEO:250', 'Altura Geopotencial @ 250 hPa [gpm]'),
 9: ('UVEL:850', 'Vento Zonal @ 850 hPa [m/s]'),
 10: ('UVEL:500', 'Vento Zonal @ 500 hPa [m/s]'),
 11: ('UVEL:250', 'Vento Zonal @ 250 hPa [m/s]'),
 12: ('VVEL:850', 'Vento Meridional @ 850 hPa [m/s]'),
 13: ('VVEL:500', 'Vento Meridional @ 500 hPa [m/s]'),
 14: ('VVEL:250', 'Vento Meridional @ 250 hPa [m/s]')}

In [ ]:
data_vars = {0: ('VT925', 'Virtual Temperature @ 925 hPa [K]'),
1: ('VT850', 'Virtual Temperature @ 850 hPa [K]'),
2: ('VT500', 'Virtual Temperature @ 500 hPa [K]'),
3: ('TM850', 'Absolute Temperature @ 850 hPa [K]'),
4: ('TM500', 'Absolute Temperature @ 500 hPa [K]'),
5: ('TM250', 'Absolute Temperature @ 250 hPa [K]'),
6: ('PSNM0', 'Pressure reduced to snm [hPa]'),
7: ('SH925', 'Specific Humidity @ 925 hPa [g/Kg]'),
8: ('SH850', 'Specific Humidity @ 850 hPa [g/Kg]'),
9: ('SH500', 'Specific Humidity @ 500 hPa [g/Kg]'),
10: ('AG925', 'Inst. Precipitable Water @ 925 hPa [Kg/m2]'),
11: ('ZG850', 'Geopotential height @ 850 hPa [gpm]'),
12: ('ZG500', 'Geopotential height @ 500 hPa [gpm]'),
13: ('ZG250', 'Geopotential height @ 250 hPa [gpm]'),
14: ('UV850', 'Zonal Wind @ 850 hPa [m/s]'),
15: ('UV500', 'Zonal Wind @ 500 hPa [m/s]'),
16: ('UV250', 'Zonal Wind @ 250 hPa [m/s]'),
17: ('VV850', 'Meridional Wind @ 850 hPa [m/s]'),
18: ('VV500', 'Meridional Wind @ 500 hPa [m/s]'),
19: ('VV250', 'Meridional Wind @  250 hPa [m/s]'),
20: ('PC000', 'TOTAL PRECIPITATION @ 1000 hPa [kg/m2/day]'),
21: ('PV001', 'CONVECTIVE PRECIPITATION @ 1000 hPa [kg/m2/day]')}


In [ ]:
data_conf = {'Starting Time': pd.datetime(2020, 6, 1, 0, 0),
 'Ending Time': pd.datetime(2020, 8, 15, 0, 0),
 'Analisys Time Step': '24',
 'Forecast Time Step': '24',
 'Forecast Total Time': '360',
 'Time Step Type': 'forward',
 'History Time': '48',
 'scantec tables': '/scripts/ensemble/SCANTEC.TESTS/tables',
 'run domain number': '1',
 'run domain lower left lat': '-80',
 'run domain lower left lon': '0',
 'run domain upper right lat': '80',
 'run domain upper right lon': '360',
 'run domain resolution dx': '0.9375000000',
 'run domain resolution dy': '0.9375000000',
 'Reference Model Name': 'GFS_0p25_5levs',
 'Reference file': '/lustre_xc50/carlos_bastarz/GFS_subset/%iy4%im2%id2%ih2/gfs.t00z.pgrb2.0p25.f000.%iy4%im2%id2%ih2.ctl',
 'Experiments': {'X126': ['BAM_TQ0126L028_9levs',
   '/lustre_xc50/carlos_bastarz/oensMB09_test_preXC50/pos/dataout/TQ0126L028/%iy4%im2%id2%ih2/NMC/GPOSNMC%iy4%im2%id2%ih2%fy4%fm2%fd2%fh2P.fct.TQ0126L028.ctl'],
  'XENM': ['BAM_TQ0126L028_9levs',
   '/lustre_xc50/carlos_bastarz/oensMB09_test_preXC50/ensmed/dataout/TQ0126L028/%iy4%im2%id2%ih2/GPOSENM%iy4%im2%id2%ih2%fy4%fm2%fd2%fh2P.fct.TQ0126L028.ctl'],
  'T126': ['BAM_TQ0126L028_9levs',
   '/lustre_xc50/carlos_bastarz/from_tupa/dados/ensemble/dsk001/oens_MB09_tupa/pos/dataout/TQ0126L028/%iy4%im2%id2%ih2/NMC/GPOSNMC%iy4%im2%id2%ih2%fy4%fm2%fd2%fh2P.fct.TQ0126L028.ctl'],
  'TENM': ['BAM_TQ0126L028_9levs',
   '/lustre_xc50/carlos_bastarz/from_tupa/dados/ensemble/dsk001/oens_MB09_tupa/ensmed/dataout/TQ0126L028/%iy4%im2%id2%ih2/GPOSENM%iy4%im2%id2%ih2%fy4%fm2%fd2%fh2P.fct.TQ0126L028.ctl']},
 'Climatology Model Name': 'AGCM_TQ0062L028_50YR_CLIMATOLOGY_18levs',
 'Climatology file': '/lustre_xc50/carlos_bastarz/climatologia/climatologia50yr.%mc.ctl',
 'Output directory': '/scripts/ensemble/SCANTEC.TESTS/dataout'}

In [ ]:
dataInicial = data_conf['Starting Time']
dataFinal = data_conf['Ending Time']
#Vars = list(map(data_vars.get,[11,12,13]))

Stats = ['RMSE']
#Exps = ['X126']
Exps = ['T126OPER']

outDir = '/home/carlos/Downloads/SCANPLOT_T11212/test/SCANTEC.TESTS/dataout/periodo/gl'

tdef = 16

xdef = 385
ydef = 171

lats = np.linspace(-90,90, ydef)
lons = np.linspace(0,360, xdef)

fnames = []

for i in [*data_vars.values()]:
    fnames.append(i[0])                           
 
nvars = len(fnames)

times = np.arange(tdef)

In [ ]:
ds_field = {}
    
for stat in Stats:
               
    dataInicial_fmt = dataInicial.strftime('%Y%m%d%H')
    dataFinal_fmt = dataFinal.strftime('%Y%m%d%H')

    for exp in Exps:
        
        #file_name = str(stat) + str(exp) + '_' + str(dataInicial_fmt) + str(dataFinal_fmt) + 'F.scan'
        file_name = str(stat) + str(exp) + '_' + str(dataInicial_fmt) + str(dataFinal_fmt) + 'F.scam'
        fname = os.path.join(outDir, file_name)
            
        lista_n = []

        if os.path.exists(fname):
                              
            dsl = []
            ds = xr.Dataset()                           
                                       
            with open(fname,'rb') as f:
                    
                for t in np.arange(tdef): 
                    
                    #f.seek(8, os.SEEK_SET) 
                    
                    for i in np.arange(nvars):
                                 
                        #f.seek(8, os.SEEK_SET) 

                        data = np.fromfile(f, dtype=np.float32, count=xdef*ydef, offset=4) 

                        #f.seek(8, os.SEEK_SET) 
                            
                        field = np.reshape(data, (xdef, ydef), order='F')  
                                                       
                        field[field == -999.9] = np.nan # substitui o valor -999.9 (undef) por NaN

                        print('time=',t,'stat=',stat,'exp=',exp,'var=',i)
                            
                        ds[fnames[i]] = (('lon','lat'), field)
                        ds.coords['lat'] = ('lat', lats)
                        ds.coords['lon'] = ('lon', lons)
                        ds.coords['time'] = [times[t]]
                                       
                        dst = ds.transpose('time', 'lat', 'lon')
                                       
                    dsl.append(dst)
                
                dsc = xr.concat(dsl, dim='time')                
                
    ds_field[ntpath.basename(str(fname))] = xr.concat(dsl, dim='time')

In [ ]:
ds_field

In [ ]:
ds_field['RMSET126OPER_20200601002020081500F.scam']

In [ ]:
(ds_field['RMSET126OPER_20200601002020081500F.scam']['VT925'].isel(time=0).min(),ds_field['RMSET126OPER_20200601002020081500F.scam']['VT925'].isel(time=0).max())

In [ ]:
ds_field['RMSET126OPER_20200601002020081500F.scam']['VT925'].isel(time=0).plot()

In [ ]:
ds_field['RMSET126OPER_20200601002020081500F.scam']['VT925'].isel(time=-1).plot()

In [ ]:
ds_field['RMSET126OPER_20200601002020081500F.scam']['VT925'].isel(time=0).plot()

In [ ]:
ds_field['RMSET126OPER_20200601002020081500F.scam']['VT925'].isel(time=-1).plot()

In [ ]:
fld = ds_field['RMSET126OPER_20200601002020081500F.scam']['VT925'].isel(time=5)

fig = plt.figure(figsize=(12,9))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines()
ax.gridlines(linewidth=0.5, color='black', alpha=0.35, linestyle='--', draw_labels=True);
#tmp.plot(ax=ax, transform=ccrs.PlateCarree(),
#         vmin=2, vmax=30, 
#         cbar_kwargs={'shrink': 0.4})
#tmp.plot(ax=ax, transform=ccrs.PlateCarree(),
#         vmin=-25, vmax=35)
fld.plot(ax=ax, transform=ccrs.PlateCarree(),vmin=-21, vmax=9)

In [ ]:
fig = plt.figure(figsize=(12,9))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines()
ax.gridlines(linewidth=0.5, color='black', alpha=0.35, linestyle='--', draw_labels=True);

fld.plot.pcolormesh(ax=ax, transform=ccrs.PlateCarree(), levels=12, vmin=-21, vmax=9, add_colorbar=True, add_labels=False)

In [ ]:
fig = plt.figure(figsize=(12,9))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines()
ax.gridlines(linewidth=0.5, color='black', alpha=0.35, linestyle='--', draw_labels=True);

fld.plot.contour(ax=ax, transform=ccrs.PlateCarree(), levels=12, vmin=-21, vmax=9)

In [ ]:
#https://ocefpaf.github.io/python4oceanographers/blog/2013/09/30/natural_earth/
    
#import cartopy.crs as ccrs
#import matplotlib.pyplot as plt
from cartopy.feature import NaturalEarthFeature, LAND, COASTLINE
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

def brazil_states(projection=ccrs.PlateCarree()):
    fig, ax = plt.subplots(figsize=(12, 9), subplot_kw=dict(projection=projection))
    ax.set_extent([-82, -32, -45, 10])
    ax.stock_img()
    ax.add_feature(LAND)
    ax.add_feature(COASTLINE)
    gl = ax.gridlines(draw_labels=True)
    gl.xlabels_top = False
    gl.ylabels_right = False
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    return fig, ax

In [ ]:
fig, ax = brazil_states()
states = NaturalEarthFeature(category='cultural', scale='110m', facecolor='none', name='admin_0_boundary_lines_land')

ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines()
ax.gridlines(linewidth=0.5, color='black', alpha=0.35, linestyle='--', draw_labels=True);

fld.plot.contour(ax=ax, transform=ccrs.PlateCarree(), levels=12, vmin=-21, vmax=9)

_ = ax.add_feature(states, edgecolor='gray')

In [ ]:
fig, ax = brazil_states()
states = NaturalEarthFeature(category='cultural', scale='110m', facecolor='none', name='admin_0_boundary_lines_land')

ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines()
ax.gridlines(linewidth=0.5, color='black', alpha=0.35, linestyle='--', draw_labels=True);

fld.plot.pcolormesh(ax=ax, transform=ccrs.PlateCarree(), levels=12, vmin=-21, vmax=9, add_colorbar=True, add_labels=False)

_ = ax.add_feature(states, edgecolor='gray')